## Scrape LSOAS

In [ ]:
import requests
import json
import pandas as pd 

In [ ]:
DEST_SIZE = 3


In [ ]:
df_lsoas = pd.read_csv("all_lsoas_with_coords.csv")
jsons = df_lsoas.to_dict(orient="records")
source = jsons[0]

In [ ]:
def make_request(source, dest_list):
    dests = ";".join([f"{c['lng']:.4f},{c['lat']:.4f}" for c in dest_list])
#     url = f"https://osrm-great-britain.apps.alpha.mojanalytics.xyz/table/v1/driving/{source['lng']:.4f},{source['lat']:.4f};{dests}?sources=0;1;2;3;4;5;6;7;8;9;10&annotations=distance,duration"
    url = f"https://osrm-great-britain.apps.alpha.mojanalytics.xyz/table/v1/driving/{source['lng']:.4f},{source['lat']:.4f};{dests}&annotations=distance,duration"
    r = requests.get(url)
    result = json.loads(r.text)
    return result


In [ ]:
make_request

In [ ]:
def result_to_table(result):
    # Process result into table
    dest1 = pd.Series(result['destinations'][1:], name="dest_result")
    dest2 = pd.Series(dest_list, name="dest_input")
    duration_seconds = pd.Series(result["durations"][0][1:], name="duration_seconds")
    distance_meters = pd.Series(result["distances"][0][1:], name="distance_meters")
    df = pd.concat([dest1,dest2, duration_seconds, distance_meters],axis=1)
    df["dest_lsoa"] = df["dest_input"].map(lambda x: x["lsoa11cd"])
    # df["dest_lat_input"] = df["dest_input"].map(lambda x: x["lat"])
    # df["dest_lng_input"] = df["dest_input"].map(lambda x: x["lng"])
    df["dest_lat"] = df["dest_result"].map(lambda x: x["location"][1])
    df["dest_lng"] = df["dest_result"].map(lambda x: x["location"][0])
    df["source_lsoa"] = source["lsoa11cd"]
    df["source_lat"] = source["lat"]
    df["source_lng"] = source["lng"]
    df.drop(["dest_result", "dest_input"], axis=1, inplace=True)
    return df

In [ ]:
%%time
dfs = []
for i in range(1000):
    start = i*500
    end = (i+1)*500
    print(f"Start: {start}, End: {end}")
    dest_list = jsons[start:end]
    if len(dest_list) == 0:
        break
    result = make_request(source, dest_list)
    df = result_to_table(result)
    print(f"Done {i}")
    dfs.append(df)
all_dfs = pd.concat(dfs)

In [ ]:
# Write results out to open data bucket 


In [ ]:
69*69

In [ ]:
result